# PROJECT 3:

## Supevised Machine Learning - Classification [African Country Recession (2000 to 2017)]

This notebook is a part of my third project from the IBM Machine Learning certificate.

The main objective of this analysis is to attempt to predict which factors contribute most to, or are most indicative of, recessions in Africa using classification. The target variable of this analysis is the ‘growthbucket’ variable and that column denotes either a "1" for "Recession" or "0" for "No_Recession". This dataset covers the period between the years 2000 and 2017.

Source Data: https://www.kaggle.com/chirin/african-country-recession-dataset-2000-to-2017

## Exploratory Data Analysis

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

In [ ]:
# We take a preliminary look at the data after loading it
filepath = "africa_recession.csv"
data = pd.read_csv(filepath)
data.head()

In [ ]:
print("Number of rows in the data:", data.shape[0])
print("Number of columns in the data:", data.shape[1])

In [ ]:
# The different columns in the dataset
print(data.columns.tolist())

In [ ]:
# The different types of data
print(data.dtypes)

In [ ]:
# We check to see if there are any missing values in each column
data.isna().sum()

In [ ]:
# Get descriptive statistics of the dataset
data.describe()

In [ ]:
# We will begin by determining if our target variable is normally distributed using a histogram
data.growthbucket.hist();

We have determined the following:

    - There are 50 columns and 486 rows in this dataset. 
    - This dataset has 2 types of data, float64 and int64. It has no object data type. 
    - There are no missing values in any of the columns.
    - We also note that the target variable is heavily skewed towards "0" (No recession).
    - We will use Log Transform to adjust the skewed data, this will decrease the effect of the outliers.

## Data Cleaning and Feature Engineering

In [ ]:
# We create a list of float colums to check for skewing
mask = data.dtypes == float
float_cols = data.columns[mask]

skew_limit = 0.75 # define a limit above which we will log transform
skew_vals = data[float_cols].skew()

In [ ]:
# We then take a look at the skewed columns
skew_cols = (skew_vals
             .sort_values(ascending=False)
             .to_frame()
             .rename(columns={0:'Skew'})
             .query('abs(Skew) > {}'.format(skew_limit)))

skew_cols

In [ ]:
# Let's look at what happens to one of these features, when we apply np.log1p visually.

# Choose a field
field = "pl_n"

# Create two "subplots" and a "figure" using matplotlib
fig, (ax_before, ax_after) = plt.subplots(1, 2, figsize=(10, 5))

# Create a histogram on the "ax_before" subplot
data[field].hist(ax=ax_before)

# Apply a log transformation (numpy syntax) to this column
data[field].apply(np.log1p).hist(ax=ax_after)

# Formatting of titles etc. for each subplot
ax_before.set(title='before np.log1p', ylabel='frequency', xlabel='value')
ax_after.set(title='after np.log1p', ylabel='frequency', xlabel='value')
fig.suptitle('Field "{}"'.format(field));

In [ ]:
# We can see that the skew transformation makes a difference to features that need it
# We then perform the skew transformation:

for col in skew_cols.index.values:
    if col == "growthbucket":
        continue
    data[col] = data[col].apply(np.log1p) 

In [ ]:
# We now apply basic feature transformations to some of the columns.
smaller_data = data.loc[:,['pl_n', 'xr', 'pop', 'ck', 'rnna', 'csh_x', 'rkna', 'pl_g', 'delta',
                      'ccon', 'cda', 'rconna', 'emp', 'cn', 'rdana', 'irr', 'pl_c', 'metals_minerals_change',
                      'rtfpna', 'forestry_change', 'csh_i', 'pl_con', 'rwtfpna',
                      'fish','csh_m', 'growthbucket']]

In [ ]:
# Now we take a look at the summary statistics of the subset data
smaller_data.describe().T

In [ ]:
smaller_data.info()

In [ ]:
# There appears to be no NaN values in the data set. Our dataset is perfectly filtered.
# We will now generate pairplot visuals to better understand the target and feature-target relationships
sns.pairplot(smaller_data, plot_kws=dict(alpha=.1, edgecolor='none'))

From the pairplot above we can see that the target variable does not seem to have a linear relationship with any of the features.
We will now:

    * Calculate the correlations between the dependent variables.
    * Create a histogram of the correlation values
    * Identify those that are most correlated (either positively or negatively).

In [ ]:
# Calculate the correlation values
feature_cols = data.columns[:-1]
corr_values = data[feature_cols].corr()
corr_values

In [ ]:
# Simplify by emptying all the data below the diagonal
tril_index = np.tril_indices_from(corr_values)
tril_index

In [ ]:
# Make the unused values NaNs
corr_array = np.array(corr_values)
corr_array[np.tril_indices_from(corr_values)] = np.nan
pd.DataFrame(corr_array)
# now we have nulled out all the values on the diagonal and below 

In [ ]:
# Now we recreate correlation pandas dataframe
corr_values = pd.DataFrame(corr_array,columns = corr_values.columns, index= corr_values.index)
corr_values.stack().to_frame().reset_index()

In [ ]:
# We stack the data and convert to a data frame
corr_values = (corr_values
               .stack()
               .to_frame()
               .reset_index()
               .rename(columns={'level_0':'feature1',
                                'level_1':'feature2',
                                0:'correlation'}))

# Get the absolute values for sorting
corr_values['abs_correlation'] = corr_values.correlation.abs()

In [ ]:
# We show a histogram of the absolute value correlations.
sns.set_context('talk')
sns.set_style('white')

ax = corr_values.abs_correlation.hist(bins=50, figsize=(12, 8))
ax.set(xlabel='Absolute Correlation', ylabel='Frequency');

- These are the most highly correlated values.
- Values are sorted by correlation going from top to bottom. We only want values from 0.8 onwards.

In [ ]:
corr_values.sort_values('correlation', ascending=False).query('abs_correlation>0.8')

We now split the data into train and test data sets before we try the Logistic Regression model.
We will be using Scikit-learn's `StratifiedShuffleSplit` to maintain the same ratio of predictor classes.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# Get the split indexes
strat_shuf_split = StratifiedShuffleSplit(n_splits=1, 
                                          test_size=0.3, 
                                          random_state=42)

train_idx, test_idx = next(strat_shuf_split.split(data[feature_cols], data.growthbucket))

# Create the dataframes
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'growthbucket']

X_test  = data.loc[test_idx, feature_cols]
y_test  = data.loc[test_idx, 'growthbucket']

In [ ]:
y_train.value_counts(normalize=True)

## Classifier Model 1: Logistic Regression

In [ ]:
# We fit a logistic regression model without any regularization using all of the features
from sklearn.linear_model import LogisticRegression

# Standard logistic regression
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
# liblinear is the "One vs the Rest" method.

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
# L1 regularized logistic regression. We used LogisticRegressionCV because it works like cv grid search.
# It will check against 10 default values of that CValue and then we can optimize from there.
lr_l1 = LogisticRegressionCV(Cs=10, cv=5, penalty='l1', max_iter=1000, solver='liblinear').fit(X_train, y_train)

# L2 regularized logistic regression
lr_l2 = LogisticRegressionCV(Cs=10, cv=5, penalty='l2', max_iter=1000, solver='liblinear').fit(X_train, y_train)

In [ ]:
# Combine all the coefficients into a dataframe
coefficients = list()

coeff_labels = ['lr', 'l1', 'l2']
coeff_models = [lr, lr_l1, lr_l2]

for lab,mod in zip(coeff_labels, coeff_models):
    coeffs = mod.coef_
    coeff_label = pd.MultiIndex(levels=[[lab], [0]], 
                                 codes=[[0], [0]])
    coefficients.append(pd.DataFrame(coeffs.T, columns=coeff_label))

coefficients = pd.concat(coefficients, axis=1)

coefficients.sample(10)

In [ ]:
# Predict the class and the probability for each model
y_pred = list()
y_prob = list()

coeff_labels = ['lr', 'l1', 'l2']
coeff_models = [lr, lr_l1, lr_l2]

for lab,mod in zip(coeff_labels, coeff_models):
    y_pred.append(pd.Series(mod.predict(X_test), name=lab))
    y_prob.append(pd.Series(mod.predict_proba(X_test).max(axis=1), name=lab))
    
y_pred = pd.concat(y_pred, axis=1)
y_prob = pd.concat(y_prob, axis=1)

y_pred.head()

In [ ]:
y_prob.head()

For each model, we calculate the following error metrics: 

* Accuracy
* Precision
* Recall
* F-score
* Confusion Matrix

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, accuracy_score

metrics = list()
cm = dict()

for lab in coeff_labels:

    # Precision, recall, f-score from the multi-class support function
    precision, recall, fscore, _ = score(y_test, y_pred[lab], average='micro')
    
    # The usual way to calculate accuracy
    accuracy = accuracy_score(y_test, y_pred[lab])
    
    
    
    # Last, the confusion matrix
    cm[lab] = confusion_matrix(y_test, y_pred[lab])
    
    metrics.append(pd.Series({'precision':precision, 'recall':recall, 
                              'fscore':fscore, 'accuracy':accuracy
                             }, 
                             name=lab))

metrics = pd.concat(metrics, axis=1)

In [ ]:
metrics

We note that the precision, recall, f score and accuracy are the same due to how we configured the Logistic Regression algorithm. 
The percentage rate for this model is 92.46% though, which is relatively high.

## Classifier Model 2: K-Nearest Neighbors

In [ ]:
data.describe().T
# We notice that the variables are on different scales

In [ ]:
# We scale the data 
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()

In [ ]:
mm.fit_transform(data)

In [ ]:
round(data.describe().T, 3)
# Now they are all on a similar scale

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score

# Estimate KNN model and report outcomes
knn = KNeighborsClassifier(n_neighbors=4)
knn = knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
# Precision, recall, f-score from the multi-class support function
print(classification_report(y_test, y_pred))
print('Accuracy score: ', round(accuracy_score(y_test, y_pred), 2))
print('F1 Score: ', round(f1_score(y_test, y_pred), 2))

In [ ]:
# Plot confusion matrix
# The blue squares are the incorrect values that we predicted
sns.set_palette(sns.color_palette(colors))
_, ax = plt.subplots(figsize=(12,12))
ax = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap=colors, annot_kws={"size": 40, "weight": "bold"})  
labels = ['False', 'True']
ax.set_xticklabels(labels, fontsize=25);
ax.set_yticklabels(labels[::-1], fontsize=25);
ax.set_ylabel('Prediction', fontsize=30);
ax.set_xlabel('Ground Truth', fontsize=30)

We noted that the accuracy score for this KNN model is 92.00%, which is lower than the Logistic Regression model's score. The F1 score is also low (15%).


## Classifier Model 3: Random Forest and Out Of Bag Errors

We will fit random forest models with a range of tree numbers and evaluate the out-of-bag error for each of these models.

In [ ]:
# Suppress warnings about too few trees from the early models
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the random forest estimator

RF = RandomForestClassifier(oob_score=True, 
                            random_state=42, 
                            warm_start=True,
                            n_jobs=-1)

oob_list = list()

# Iterate through all of the possibilities for 
# number of trees
for n_trees in [15, 20, 30, 40, 50, 100, 150, 200, 300, 400]:
    
    # Use this to set the number of trees
    RF.set_params(n_estimators=n_trees)

    # Fit the model
    RF.fit(X_train, y_train)

    # Get the oob error
    oob_error = 1 - RF.oob_score_
    
    # Store it
    oob_list.append(pd.Series({'n_trees': n_trees, 'oob': oob_error}))

rf_oob_df = pd.concat(oob_list, axis=1).T.set_index('n_trees')

rf_oob_df

In [ ]:
sns.set_context('talk')
sns.set_style('white')

ax = rf_oob_df.plot(legend=False, marker='o', figsize=(14, 7), linewidth=5)
ax.set(ylabel='out-of-bag error');

The error looks like it has stabilized at around 40 trees. This is where it plateaus. We will now use extra randomized trees (`ExtraTreesClassifier`). We will then compare the out-of-bag errors for the two different types of models.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

# Initialize the random forest estimator

EF = ExtraTreesClassifier(oob_score=True, 
                          random_state=42, 
                          warm_start=True,
                          bootstrap=True,
                          n_jobs=-1)

oob_list = list()

# Iterate through all of the possibilities for 
# number of trees
for n_trees in [15, 20, 30, 40, 50, 100, 150, 200, 300, 400]:
    
    # Use this to set the number of trees
    EF.set_params(n_estimators=n_trees)
    EF.fit(X_train, y_train)

    # oob error
    oob_error = 1 - EF.oob_score_
    oob_list.append(pd.Series({'n_trees': n_trees, 'oob': oob_error}))

et_oob_df = pd.concat(oob_list, axis=1).T.set_index('n_trees')

et_oob_df

In [ ]:
# We will combine the two dataframes into a single one for easier plotting.
oob_df = pd.concat([rf_oob_df.rename(columns={'oob':'RandomForest'}),
                    et_oob_df.rename(columns={'oob':'ExtraTrees'})], axis=1)

oob_df

In [ ]:
sns.set_context('talk')
sns.set_style('white')

ax = oob_df.plot(marker='o', figsize=(14, 7), linewidth=5)
ax.set(ylabel='out-of-bag error');

We note that the extra randomized trees model performs better than the random forest model.

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

cr = classification_report(y_test, y_pred)
print(cr)

score_df = pd.DataFrame({'accuracy': accuracy_score(y_test, y_pred),
                         'precision': precision_score(y_test, y_pred),
                         'recall': recall_score(y_test, y_pred),
                         'f1': f1_score(y_test, y_pred),
                         'auc': roc_auc_score(y_test, y_pred)},
                         index=pd.Index([0]))

print(score_df)

We note that the precision, f1, support score and recall for "0" are high but low for "1". The general accuracy score is 91.77% which is lower than the Logistic Regression and the KNN models' scores.

In [ ]:
sns.set_context('talk')
cm = confusion_matrix(y_test, y_pred)
_, ax = plt.subplots(figsize=(12,12))
ax = sns.heatmap(cm, annot=True, fmt='d', cmap=colors, annot_kws={"size": 40, "weight": "bold"})

labels = ['False', 'True']
ax.set_xticklabels(labels, fontsize=25);
ax.set_yticklabels(labels[::-1], fontsize=25);
ax.set_ylabel('Prediction', fontsize=30);
ax.set_xlabel('Ground Truth', fontsize=30)

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix
sns.set_context('talk')

fig, axList = plt.subplots(ncols=2)
fig.set_size_inches(16, 8)

# Get the probabilities for each of the two categories
y_prob = model.predict_proba(X_test)

# Plot the ROC-AUC curve
ax = axList[0]

fpr, tpr, thresholds = roc_curve(y_test, y_prob[:,1])
ax.plot(fpr, tpr, color=colors[0], linewidth=5)
# It is customary to draw a diagonal dotted line in ROC plots.
# This is to indicate completely random prediction. Deviation from this
# dotted line towards the upper left corner signifies the power of the model.
ax.plot([0, 1], [0, 1], ls='--', color='black', lw=.3)
ax.set(xlabel='False Positive Rate',
       ylabel='True Positive Rate',
       xlim=[-.01, 1.01], ylim=[-.01, 1.01],
       title='ROC curve')
ax.grid(True)

# Plot the precision-recall curve
ax = axList[1]

precision, recall, _ = precision_recall_curve(y_test, y_prob[:,1])
ax.plot(recall, precision, color=colors[1], linewidth=5)
ax.set(xlabel='Recall', ylabel='Precision',
       xlim=[-.01, 1.01], ylim=[-.01, 1.01],
       title='Precision-Recall curve')
ax.grid(True)

plt.tight_layout()

We note that the ROC curve is far from the top left corner so the test is not efficient. This makes sense since this data is imbalanced. The Precision recall curve is also not efficient for this data. This is all due to the data's accuracy being low for the "1" class as noted before.

## Conclusion

In [ ]:
feature_imp = pd.Series(model.feature_importances_, index=feature_cols).sort_values(ascending=False)

ax = feature_imp.plot(kind='bar', figsize=(16, 6))
ax.set(ylabel='Relative Importance');
ax.set(ylabel='Feature');

The above bar chart shows us which features are the most important when it comes to determining which features have the most influence on the target variable. In this case it is the "rwtfpna" (TFP at constant national prices (2011=1) feature. This feature is basically the portion of output that is not explained by the amount of inputs used in production for the reference year 2011.

We can also conclude that the best model to use on this dataset is the Logistic Regression model. It has the highest accuracy score, although not by much as compared to the KNN and Random Trees models. 